## Landscape

In [3]:
import os
import subprocess

def has_landscape_config():
    # Check for a specific file or directory associated with the landscape module
    landscape_files = ['/etc/landscape/client.conf', '/etc/landscape/client.conf.d/']
    
    for file_or_dir in landscape_files:
        if os.path.exists(file_or_dir):
            return True
    
    return False

def search_landscape_config_entries():
    # Search for specific entries in configuration files
    landscape_keywords = ['landscape', 'some_other_keyword']
    config_files = ['/etc/landscape/client.conf', '/etc/landscape/client.conf.d/']

    for file_path in config_files:
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                for line in f:
                    if any(keyword in line for keyword in landscape_keywords):
                        return True
    return False

def is_landscape_package_installed():
    # Use dpkg to check if the landscape package is installed
    result = subprocess.run("dpkg -l | grep landscape", shell=True, stdout=subprocess.PIPE, text=True)
    return "landscape" in result.stdout

def gather() -> dict:
    return {
        "Landscape": {
            "has_landscape_config": has_landscape_config(),
            "has_landscape_config_entries": search_landscape_config_entries(),
            "is_landscape_package_installed": is_landscape_package_installed(),
        }
    }
    
gather()

{'Landscape': {'has_landscape_config': False,
  'has_landscape_config_entries': False,
  'is_landscape_package_installed': False}}

## APT Repositories

In [20]:
import dataclasses
from pprint import pprint
from typing import Optional
import os
import subprocess

@dataclasses.dataclass
class AptRepository:
    archive_type: str
    options: Optional[dict[str, list[str]]]
    uri: str
    suite: str
    components: Optional[list]

def get_apt_repositories() -> list[AptRepository]:
    # APT configuration files
    sources_list_path = '/etc/apt/sources.list'
    sources_list_d_path = '/etc/apt/sources.list.d/'

    repositories = []

    # Read main sources.list
    with open(sources_list_path, 'r') as sources_list_file:
        for line in sources_list_file:
            if line.startswith('deb'):
                repositories.append(parse_repository_line(line.strip()))

    # Read files in sources.list.d directory
    for filename in os.listdir(sources_list_d_path):
        file_path = os.path.join(sources_list_d_path, filename)
        with open(file_path, 'r') as sources_list_d_file:
            for line in sources_list_d_file:
                if line.startswith('deb'):
                    repositories.append(parse_repository_line(line.strip()))

    return repositories

def parse_repository_line(line: str) -> AptRepository:
    repo_info = {}
    import re 
    repo_info["archive_type"] = line.split()[0]
    # if options exist
    if re.findall(r'\[.*?\]', line):
        repo_info["options"] = {}
        options_str = re.findall(r'\[(.*?)\]', line)[0]
        # parse each option and add to dict
        for option in options_str.strip().split():
            key, value = option.split("=")
            # if value is comma separated, split into list
            repo_info["options"][key] = value.split(",")
        # replace options string with empty string to make parsing rest of line easier
        line = line.replace(f"[{options_str}]", "")
    else:
        repo_info["options"] = None
    repo_info["uri"] = line.split()[1]
    repo_info["suite"] = " ".join(line.split()[2:min(4, len(line.split()))])
    if len(line.split()) > 3:
        repo_info["components"] = line.split()[3:]
    else:
        repo_info["components"] = None

    return AptRepository(**repo_info)

def get_apt_repositories_with_commands():
    # Use APT commands to get repositories
    result_main = subprocess.run("cat /etc/apt/sources.list", shell=True, stdout=subprocess.PIPE, text=True)
    result_d = subprocess.run("cat /etc/apt/sources.list.d/*", shell=True, stdout=subprocess.PIPE, text=True)

    repositories = result_main.stdout.strip().split('\n')
    repositories.extend(result_d.stdout.strip().split('\n'))

    return repositories

def gather() -> dict:
    return {
        "AptRepositories": {
            "repositories": get_apt_repositories(),
            "repositories_with_commands": get_apt_repositories_with_commands(),
        }
    }

pprint(gather()["AptRepositories"]["repositories"])

[AptRepository(archive_type='deb',
               options=None,
               uri='https://ppa.launchpadcontent.net/linrunner/tlp/ubuntu/',
               suite='jammy',
               components=['main']),
 AptRepository(archive_type='deb',
               options=None,
               uri='https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu/',
               suite='jammy',
               components=['main']),
 AptRepository(archive_type='deb',
               options={'arch': ['amd64'],
                        'signed-by': ['/etc/apt/keyrings/docker.gpg']},
               uri='https://download.docker.com/linux/ubuntu',
               suite='jammy',
               components=['stable']),
 AptRepository(archive_type='deb',
               options=None,
               uri='https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu/',
               suite='jammy',
               components=['main']),
 AptRepository(archive_type='deb',
               options={'arch': ['amd64']},
       

## APT Packages

In [16]:
from pprint import pprint
import re
import subprocess
def get_installed_apt_packages_from_dpkg():
    # Run dpkg -l command to get a list of installed packages
    result = subprocess.run("dpkg -l", shell=True, stdout=subprocess.PIPE, text=True)

    # Parse the output to extract package names
    installed_packages = []
    lines = result.stdout.strip().split('\n')
    for line in lines[5:]:  # Skip the header lines
        package_info = line.split()
        if len(package_info) >= 2:
            package_name = package_info[1]
            installed_packages.append(package_name)

    return installed_packages

def get_installed_apt_packages_from_apt():
    result = subprocess.run("apt-mark showmanual", shell=True, stdout=subprocess.PIPE, text=True)
    return result.stdout.strip().split('\n')

def get_installed_apt_packages_from_history():
    result = subprocess.run("zgrep 'Commandline: apt' /var/log/apt/history.log /var/log/apt/history.log.*.gz", shell=True, stdout=subprocess.PIPE, text=True)
    # pprint(result.stdout.strip().split('\n'))
    # return result.stdout.strip().split('\n')
    packages = []
    for line in result.stdout.strip().split('\n'):
        if re.findall(r"Commandline: apt(-get)? install ", line):
            parsed_packages = [
                item for item in line.split("install", maxsplit=1)[1].split() 
                if not item.startswith("-")
            ]
            packages += parsed_packages
    packages.sort()
    return packages

def gather():

    # Return the results
    return {
        "AptPackages": {
            "packages_dpkg":  get_installed_apt_packages_from_dpkg(),
            "packages_apt": get_installed_apt_packages_from_apt(),
            "packages_apt_history": get_installed_apt_packages_from_history(),
        }
    }

result = gather()
print(len(result['AptPackages']['packages_dpkg']))
print(len(result['AptPackages']['packages_apt']))
print(len(result['AptPackages']['packages_apt_history']))
pprint(result['AptPackages']['packages_apt_history'])

2625
445
228
['amd-ppt-bin',
 'apt-transport-https',
 'awscli',
 'azure-cli',
 'build-essential',
 'bzr',
 'ca-certificates',
 'casper',
 'chromium-browser',
 'clang',
 'cloud-image-utils',
 'containerd.io',
 'curl',
 'curl',
 'devscripts',
 'distinst',
 'docker',
 'docker-buildx-plugin',
 'docker-ce',
 'docker-ce-cli',
 'docker-compose-plugin',
 'expect',
 'fonts-arphic-ukai',
 'fonts-arphic-ukai',
 'fonts-arphic-uming',
 'fonts-arphic-uming',
 'fonts-noto-cjk',
 'fonts-noto-cjk',
 'fonts-noto-cjk',
 'fonts-noto-cjk-extra',
 'fonts-noto-cjk-extra',
 'fonts-noto-cjk-extra',
 'fonts-noto-core',
 'fonts-noto-ui-core',
 'gconf2',
 'gnome-tweaks',
 'gnome-user-docs-de',
 'gnome-user-docs-es',
 'gnome-user-docs-fr',
 'gnome-user-docs-it',
 'gnome-user-docs-ja',
 'gnome-user-docs-pt',
 'gnome-user-docs-ru',
 'gnome-user-docs-zh-hans',
 'gnome-user-docs-zh-hans',
 'gnupg',
 'gnupg',
 'gnupg',
 'gparted',
 'hunspell-ar',
 'hunspell-de-at-frami',
 'hunspell-de-ch-frami',
 'hunspell-de-de-frami'

## Users Config 

- get users config
- is user sudo?
- what groups is user in?
- what is user's shell?
- get content of user's bashrc or zshrc 
  - get zshrc completely duplicateable?  
- get user's ssh keys so they can later be used with ssh-import-id (ask more about import-id)

## Pro Client Config
- ask table flip server for what can actually be captured and then later configured using cloud-init